In [ ]:
!pip install opencv-python

In [ ]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-9k8agitt
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-9k8agitt
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp36-none-any.whl size=101066 sha256=b2a6a4f55bf594709e71bd59e9422d07db9bebe8f553571ee58b68388d80488a
  Stored in directory: /tmp/pip-ephem-wheel-cache-htsqastg/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib


In [ ]:
from keras.optimizers import Adam
from keras.initializers import RandomNormal
from keras.models import Model
from keras.models import Input
from keras.models import load_model
from keras.layers import Conv2D, Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Activation
from keras.layers import Concatenate
from keras.layers import BatchNormalization
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from keras.utils.vis_utils import plot_model
import numpy as np
import os
import datetime
import matplotlib.pyplot as plt
import sys
import tensorflow as tf

In [ ]:
class Gan():
    def __init__(self):
        self.img_rows = 128
        self.img_cols = 128
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)

        optimizer = Adam(lr=0.0002, beta_1=0.5, beta_2=0.999)

        self.dataset_name = 'simps2van'
        self.data_loader = DataLoader(dataset=self.dataset_name,
                                      image_shape=(self.img_rows, self.img_cols))

        # Calculate output shape of Discriminator (PatchGAN)
        patch = int(self.img_rows / 2**4)
        self.disc_patch = (patch, patch, 1)

        # build and compile discriminator: A -> [real/fake]
        self.d_model_A = self.build_discriminator()
        #self.d_model_A = load_model('/content/drive/MyDrive/Colab Notebooks/dlschool/16. HWGAN/models/discriminatorA.h5',custom_objects={'InstanceNormalization': InstanceNormalization}, compile=False)
        self.d_model_A.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])
        
        
        # build and compile discriminator: B -> [real/fake]
        self.d_model_B = self.build_discriminator()
        #self.d_model_B = load_model('/content/drive/MyDrive/Colab Notebooks/dlschool/16. HWGAN/models/discriminatorB.h5',custom_objects={'InstanceNormalization': InstanceNormalization}, compile=False)
        self.d_model_B.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])
        
        

        # build generator: A -> B
        self.g_AtoB = self.build_generator()
        #self.g_AtoB = load_model('/content/drive/MyDrive/Colab Notebooks/dlschool/16. HWGAN/models/generatorA2B.h5',custom_objects={'InstanceNormalization': InstanceNormalization}, compile=False) 
        
        # build generator: B -> A
        self.g_BtoA = self.build_generator()
        #self.g_BtoA = load_model('/content/drive/MyDrive/Colab Notebooks/dlschool/16. HWGAN/models/generatorB2A.h5',custom_objects={'InstanceNormalization': InstanceNormalization}, compile=False)
        

        # define input images for both domains
        img_A = Input(shape=self.img_shape)
        img_B = Input(shape=self.img_shape)

        # identity element
        fake_B = self.g_AtoB(img_A)
        # forward cycle
        fake_A = self.g_BtoA(img_B)
        # backward cycle
        back_to_A = self.g_BtoA(fake_B)
        back_to_B = self.g_AtoB(fake_A)
        # Identity mapping of images
        img_A_id = self.g_BtoA(img_A)
        img_B_id = self.g_AtoB(img_B)


        # For the combined model we will only train the generators
        self.d_model_A.trainable = False
        self.d_model_B.trainable = False

        # Discriminators determines validity of translated images
        valid_A = self.d_model_A(fake_A)
        valid_B = self.d_model_B(fake_B)

        # define a composite model for updating generators by adversarial and cycle loss
        self.composite_model = Model(inputs=[img_A, img_B], outputs=[valid_A, valid_B, back_to_A, back_to_B, img_A_id, img_B_id])

        # compile model with weighting of least squares loss and L1 loss
        self.composite_model.compile(loss=['mse', 'mse',
                                           'mae', 'mae', 
                                           'mae', 'mae'],
                                     loss_weights=[1, 1, 10, 10, 1, 1], 
                                     optimizer = optimizer)

    def build_discriminator(self):
        'Define the Discriminator model'
        # initialization weight
        init = RandomNormal(stddev=0.02)
        # input_image
        in_image = Input(shape=self.img_shape)
        # optimizer use
        optimizer = Adam(lr=0.0002, beta_1=0.5, beta_2=0.999)

        def disc_layer(in_image, out_channels, strides=(2,2), instance_norm=True, initializer=init):
            'Layer for building Discriminator'
            d = Conv2D(out_channels, kernel_size=(4,4), strides=strides, padding='same', kernel_initializer=initializer)(in_image)
            if instance_norm:
                d = InstanceNormalization(axis=-1)(d)
            d = LeakyReLU(alpha=0.2)(d)
            return d

        # convolutions layers
        d = disc_layer(in_image, 64, instance_norm=False)
        d = disc_layer(d, 128)
        d = disc_layer(d, 256)
        d = disc_layer(d, 512)
        d = disc_layer(d, 512, strides=(1,1))

        # output layer
        out = Conv2D(1, 4, padding='same', kernel_initializer=init)(d)

        # define model
        model = Model(in_image, out)
        return model

    def build_generator(self, n_resnet=9):
        'Define the Generator model'
        # initialization weight
        init = RandomNormal(stddev=0.02)
        # input_image
        in_image = Input(shape=self.img_shape)

        def resnet_block(n_filters, input_layer, initializer=init):
            'Residual Connection block for building generator'

            # first layer
            rb = Conv2D(filters=n_filters, kernel_size=3, padding='same', kernel_initializer=initializer)(input_layer)
            rb = InstanceNormalization(axis=-1)(rb)
            rb = Activation('relu')(rb)

            # second layer
            rb = Conv2D(filters=n_filters, kernel_size=3, padding='same', kernel_initializer=initializer)(rb)
            rb = InstanceNormalization(axis=-1)(rb)

            # residual connection 
            rb = Concatenate()([rb, input_layer])
            return rb

        def main_block(input_layer, in_features=64, downsampling=True, initializer=init):
            'Downsampling or Upsampling block'
            if downsampling == True:
                out_features = in_features*2
                g = Conv2D(out_features, kernel_size=3, strides=(2,2), padding='same', kernel_initializer=initializer)(input_layer)
            elif downsampling == False:
                out_features = in_features//2
                g = Conv2DTranspose(out_features, kernel_size=3, strides=(2,2), padding='same', kernel_initializer=initializer)(input_layer)

            
            g = InstanceNormalization(axis=-1)(g)
            g = Activation('relu')(g)
            return g

        # c7s1-64
        g = Conv2D(64, (7,7), padding='same', kernel_initializer=init)(in_image)
        g = InstanceNormalization(axis=-1)(g)
        g = Activation('relu')(g)
    
        # d128     
        g = main_block(input_layer=g, in_features=64, downsampling=True)
        # d256
        g = main_block(input_layer=g, in_features=128, downsampling=True)

        # R256
        for _ in range(n_resnet):
            g = resnet_block(256, g)

        # u128
        g = main_block(input_layer=g, in_features=256, downsampling=False)
        # u64
        g = main_block(input_layer=g, in_features=128, downsampling=False)

        # c7s1-3
        g = Conv2D(3, (7,7), padding='same', kernel_initializer=init)(g)
        g = InstanceNormalization(axis=-1)(g)
        out_image = Activation('tanh')(g)
    
        model = Model(in_image, out_image)
        return model

    
    def train(self, epochs, batch_size, sample_interval=5):
        
        start_time = datetime.datetime.now()

        # Adversarial loss ground truths
        valid = np.ones((batch_size,) + self.disc_patch)
        fake = np.zeros((batch_size,) + self.disc_patch)
        
        # Train loop
        for epoch in range(epochs):
            for batch, (img_A, img_B) in enumerate(self.data_loader.load_batch(batch_size)):

                # Discriminators
                #-----------------------------------------
                # translate images to the other domain
                fake_B = self.g_AtoB.predict(img_A)
                fake_A = self.g_BtoA.predict(img_B)
                
                # train Discriminator A
                d_model_A_loss_real = self.d_model_A.train_on_batch(img_A, valid)
                d_model_A_loss_fake = self.d_model_A.train_on_batch(fake_A, fake)
                d_model_A_loss = 0.5 * np.add(d_model_A_loss_real, d_model_A_loss_fake)
                # train Discriminator B
                d_model_B_loss_real = self.d_model_B.train_on_batch(img_B, valid)
                d_model_B_loss_fake = self.d_model_B.train_on_batch(fake_B, fake)
                d_model_B_loss = 0.5 * np.add(d_model_B_loss_real, d_model_B_loss_fake)

                # total loss of Discriminators
                total_d_loss = 0.5 * np.add(d_model_A_loss, d_model_B_loss)



                # Generators
                #---------------------------------------
                g_loss = self.composite_model.train_on_batch([img_A, img_B], [valid, valid, img_A, img_B, img_A, img_B])

                elapsed_time = datetime.datetime.now() - start_time

                # Plot the progress
                print ("[Epoch %d/%d] [Batch %d/%d] [D loss: %f, acc: %3d%%] [G loss: %05f, adv: %05f, recon: %05f, id: %05f] time: %s " \
                                                                        % ( epoch, epochs,
                                                                            batch, self.data_loader.n_batches,
                                                                            total_d_loss[0], 100*total_d_loss[1],
                                                                            g_loss[0],
                                                                            np.mean(g_loss[1:3]),
                                                                            np.mean(g_loss[3:5]),
                                                                            np.mean(g_loss[5:6]),
                                                                            elapsed_time))
                

                # If at save interval => save generated image samples
                if batch % sample_interval == 0:
                    self.sample_images(epoch, batch)

                if batch % 80 == 0:
                    #self.g_AtoB.save('/content/drive/MyDrive/Colab Notebooks/dlschool/16. HWGAN/models/generatorA2B_epoch_%d.h5' % epoch)
                    #self.g_BtoA.save('/content/drive/MyDrive/Colab Notebooks/dlschool/16. HWGAN/models/generatorB2A_epoch_%d.h5' % epoch)
                    #self.d_model_A.save('/content/drive/MyDrive/Colab Notebooks/dlschool/16. HWGAN/models/discriminatorA_epoch_%d.h5' % epoch)
                    #self.d_model_B.save('/content/drive/MyDrive/Colab Notebooks/dlschool/16. HWGAN/models/discriminatorB_epoch_%d.h5' % epoch)
                    self.g_AtoB.save('/content/drive/MyDrive/Colab Notebooks/dlschool/16. HWGAN/models2/generatorA2B.h5')
                    self.g_BtoA.save('/content/drive/MyDrive/Colab Notebooks/dlschool/16. HWGAN/models2/generatorB2A.h5')
                    self.d_model_A.save('/content/drive/MyDrive/Colab Notebooks/dlschool/16. HWGAN/models2/discriminatorA.h5')
                    self.d_model_B.save('/content/drive/MyDrive/Colab Notebooks/dlschool/16. HWGAN/models2/discriminatorB.h5')
                    
                    #if epoch > 3:
                    #    os.remove('/content/drive/MyDrive/Colab Notebooks/dlschool/16. HWGAN/models/generatorA2B_epoch_%d.h5' % epoch-2)
                    #    os.remove('/content/drive/MyDrive/Colab Notebooks/dlschool/16. HWGAN/models/generatorB2A_epoch_%d.h5' % epoch-2)
                    #    os.remove('/content/drive/MyDrive/Colab Notebooks/dlschool/16. HWGAN/models/discriminatorA_epoch_%d.h5' % epoch-2)
                    #    os.remove('/content/drive/MyDrive/Colab Notebooks/dlschool/16. HWGAN/models/discriminatorB_epoch_%d.h5' % epoch-2)
                    

                    

                    

    def sample_images(self, epoch, batch):
        os.makedirs('/content/drive/MyDrive/Colab Notebooks/dlschool/16. HWGAN/images/%s' % self.dataset_name, exist_ok=True)
        r, c = 2, 3

        imgs_A = self.data_loader.load_dataset(domain="A", batch_size=1, test=True)
        imgs_B = self.data_loader.load_dataset(domain="B", batch_size=1, test=True)


        # Translate images to the other domain
        fake_B = self.g_AtoB.predict(imgs_A)
        fake_A = self.g_BtoA.predict(imgs_B)
        # Translate back to original domain
        reconstr_A = self.g_BtoA.predict(fake_B)
        reconstr_B = self.g_AtoB.predict(fake_A)

        gen_imgs = np.concatenate([imgs_A, fake_B, reconstr_A, imgs_B, fake_A, reconstr_B])

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        titles = ['Original', 'Translated', 'Reconstructed']
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt])
                axs[i, j].set_title(titles[j])
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("/content/drive/MyDrive/Colab Notebooks/dlschool/16. HWGAN/images/%s/%d_%d.png" % (self.dataset_name, epoch, batch))
        plt.close()




In [ ]:
from glob import glob
import numpy as np
import cv2
from matplotlib.pyplot import imread
class DataLoader():
    def __init__(self, dataset, image_shape=(128,128)):
        self.dataset_name = dataset
        self.image_shape = image_shape

    def read_image(self, image_path):
        'Read an image from a file as an array'
        image = imread(image_path, 'RGB').astype(np.float)
        return image

    def load_image(self, image_path):
        'Read and prepare image'
        image = self.read_image(image_path)
        image = cv2.resize(image, self.image_shape)
        image = image/127.5 - 1.
        return image

    def load_dataset(self, domain, batch_size=12, test=False):
        'Load data from dataset folders, and prepare image for batch'
        # Select the folder:  'testA' / 'trainB'...
        image_shape = (128,128)
        if test:
            data_split = f'test{domain}'
        else:
            data_split = f'train{domain}'

        path = glob(f'/content/drive/MyDrive/Colab Notebooks/dlschool/16. HWGAN/cyclegan_datasets/{self.dataset_name}/{data_split}/*')    

        # define batch
        batch = np.random.choice(a=path, size=batch_size)

        # prepare images
        images = []
        for image in batch:
            # read
            image = self.read_image(image)
            # resize
            if test:
                image = cv2.resize(image, image_shape)
            else:
                image = cv2.resize(image, image_shape)
                # flip
                if np.random.random() > 0.5:
                    image = np.fliplr(img)
            
            images.append(image)
        # normalization
        images = np.array(images)/127.5 - 1.
        
        return images

    


    def load_batch(self, batch_size=12, image_shape=(128,128)):
        # define path
        path_A = glob('/content/drive/MyDrive/Colab Notebooks/dlschool/16. HWGAN/cyclegan_datasets/%s/trainA/*' % (self.dataset_name))
        path_B = glob('/content/drive/MyDrive/Colab Notebooks/dlschool/16. HWGAN/cyclegan_datasets/%s/trainB/*' % (self.dataset_name))

        # how many batches are in the dataset
        self.n_batches = int(min(len(path_A), len(path_B)) / batch_size)
        total_samples = self.n_batches * batch_size


        path_A = np.random.choice(path_A, total_samples, replace=False)
        path_B = np.random.choice(path_B, total_samples, replace=False)

        for i in range(self.n_batches-1):
            batch_A = path_A[i*batch_size:(i+1)*batch_size]
            batch_B = path_B[i*batch_size:(i+1)*batch_size]
            imgs_A, imgs_B = [], []
            for img_A, img_B in zip(batch_A, batch_B):
                img_A = self.read_image(img_A)
                img_B = self.read_image(img_B)

                img_A = cv2.resize(img_A, image_shape)
                img_B = cv2.resize(img_B, image_shape)

                if np.random.random() > 0.5:
                        img_A = np.fliplr(img_A)
                        img_B = np.fliplr(img_B)

                imgs_A.append(img_A)
                imgs_B.append(img_B)

            imgs_A = np.array(imgs_A)/127.5 - 1.
            imgs_B = np.array(imgs_B)/127.5 - 1.

            yield imgs_A, imgs_B

In [ ]:
gan = Gan()
gan.train(epochs=500, batch_size=8, sample_interval=60)

[Epoch 0/500] [Batch 0/12] [D loss: 2.190488, acc:  36%] [G loss: 20.246498, adv: 2.914340, recon: 0.656055, id: 0.631148] time: 0:00:58.829794 
[Epoch 0/500] [Batch 1/12] [D loss: 2.712001, acc:  49%] [G loss: 31.118229, adv: 8.587090, recon: 0.634453, id: 0.633231] time: 0:01:43.976036 
[Epoch 0/500] [Batch 2/12] [D loss: 3.011987, acc:  48%] [G loss: 33.410091, adv: 9.751606, recon: 0.632992, id: 0.604017] time: 0:02:04.798463 
[Epoch 0/500] [Batch 3/12] [D loss: 4.814436, acc:  51%] [G loss: 28.244402, adv: 7.490218, recon: 0.602932, id: 0.560192] time: 0:02:29.400377 
[Epoch 0/500] [Batch 4/12] [D loss: 3.442484, acc:  47%] [G loss: 37.104618, adv: 11.759742, recon: 0.615547, id: 0.631935] time: 0:02:48.529202 
[Epoch 0/500] [Batch 5/12] [D loss: 3.915137, acc:  50%] [G loss: 24.473717, adv: 5.769480, recon: 0.582875, id: 0.633339] time: 0:03:13.218608 
[Epoch 0/500] [Batch 6/12] [D loss: 3.212097, acc:  49%] [G loss: 23.317858, adv: 5.649235, recon: 0.536524, id: 0.628715] time: 

KeyboardInterrupt: ignored